<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/experimental/colab_drive_mount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ColabでGoogleDriveをマウントする方法を説明します。

## 背景
Colabでは、ランタイムを削除するとDisk上のファイルがクリアされます。RusyBotはシステム上にSqlite3のDBをつくっているためランタイム削除のたびにDBがクリアされることになります。

そこで、DBをGoogleDriveに置く方法を説明します。

BinanceConfig.db_base_dirでDB保存先を変更する方法は、ローカルPCで余裕のあるディスクにDBを構築したい場合にも有効です。

注：version　0.2.15 から環境変数 `RBOT_DB_ROOT`に設定するように変更になります。

In [ ]:
! pip install rbot

In [ ]:
from rbot import BinanceConfig
from rbot import BinanceMarket

In [ ]:
# Google Driveをマウントする
from google.colab import drive

drive.mount('/content/drive')

# DBの保存先をGoogle Driveに変更する
config = BinanceConfig.BTCUSDT
config.db_base_dir = '/content/drive/MyDrive'    # MyDrive以下を指定しましたが適宜変更ください

# version 0.2.15から環境変数へ設定するように変更します。
# 以下のように設定してください。
import os
os.environ['RBOT_DB_ROOT'] = '/content/drive/MyDrive'    # MyDrive以下を指定しましたが適宜変更ください

In [ ]:
# Binanceのインスタンスを作成する
binance = BinanceMarket(config)

# 具体的に作られたDBのパスを表示
binance.file_name

In [ ]:
# データをダウンロードする。１０日分は案外時間がかかります。
binance.download(ndays=10, verbose=True, archive_only=True)

In [ ]:
# ダウンロード状況を確認（実際にダウンロードされた日数は、まだアーカイブされていない日があるため１日減ることがあります）
binance

## 折角なので分析

kashiharaさんの
「国内取引所と海外取引所の短期的値動きの変化」
https://note.com/kashihara1/n/nb3424848a968

を参考にボラティリティ（＝振れ幅？）を分析してみます。

元記事では、おそらくボラティリティが年々小さくなっていてBotにはつらい状況になってきているということを分析されているのだと理解しました。

以下簡単に直近の数字をしらべてみました。

In [ ]:
# 1分足
from rbot import NOW
from rbot import DAYS

# 9日分の４本足を作成します。
ohlcv = binance.ohlcv(start_time=NOW()-DAYS(9), end_time=0, window_sec=60)

In [ ]:
# ohlcvにPorlasのDataFrameで格納されています。
ohlcv.head()

In [ ]:
# pandasに変換して分析することも可能です。
ohlcv_df = ohlcv.to_pandas(use_pyarrow_extension_array=True)

ohlcv_df.head()

In [ ]:
# HighとLowの差分を追加しましょう
range = (ohlcv.get_column('high') - ohlcv.get_column('low')).alias('range')
ohlcv = ohlcv.with_columns(range)

# 差分の終値に対する比率を追加しましょう
range_rate = (range / ohlcv['close']).alias('range_rate')
ohlcv = ohlcv.with_columns(range_rate)

ohlcv.head()


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(ohlcv['timestamp'], ohlcv['range_rate'])
plt.show()

In [ ]:
# range_rateの概要（統計値）を表示する
ohlcv['range_rate'].describe()

In [ ]:
# rangeの概要（統計値）を表示する
ohlcv['range'].describe()

## まとめ
2023年12月21日で試したみたところ、１本足での変化率は平均0.06%ぐらい。具体的な値としては27ドルぐらい。

これは１分の間のポジションをオープンしクローズする高頻度だと完璧に変化をあてても２７ドルしか鞘がとれないということ。0.06%のうち、手数料で往復0.02%とられることを考えると残り3分の2になって20ドルの鞘も難しいということらしい。

もうすこし長期でかんがてみようかなあ。